# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8a7c15cfd0>
├── 'a' --> tensor([[-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745]])
└── 'x' --> <FastTreeValue 0x7f8a7c15c9a0>
    └── 'c' --> tensor([[-1.8615,  1.6310,  0.0339,  0.1787],
                        [ 0.1815,  1.4565,  1.5177,  1.9505],
                        [ 1.0834,  2.7347, -0.6714, -0.6708]])

In [4]:
t.a

tensor([[-0.5418,  1.7838, -0.7986],
        [-1.1647,  0.0565,  0.3745]])

In [5]:
%timeit t.a

72 ns ± 0.0647 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8a7c15cfd0>
├── 'a' --> tensor([[ 1.2392, -2.4594,  0.2019],
│                   [-2.2108, -0.4363,  0.1960]])
└── 'x' --> <FastTreeValue 0x7f8a7c15c9a0>
    └── 'c' --> tensor([[-1.8615,  1.6310,  0.0339,  0.1787],
                        [ 0.1815,  1.4565,  1.5177,  1.9505],
                        [ 1.0834,  2.7347, -0.6714, -0.6708]])

In [7]:
%timeit t.a = new_value

77 ns ± 0.0677 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5418,  1.7838, -0.7986],
               [-1.1647,  0.0565,  0.3745]]),
    x: Batch(
           c: tensor([[-1.8615,  1.6310,  0.0339,  0.1787],
                      [ 0.1815,  1.4565,  1.5177,  1.9505],
                      [ 1.0834,  2.7347, -0.6714, -0.6708]]),
       ),
)

In [10]:
b.a

tensor([[-0.5418,  1.7838, -0.7986],
        [-1.1647,  0.0565,  0.3745]])

In [11]:
%timeit b.a

66.5 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.7516,  1.2288, -1.4041],
               [ 0.8290, -0.2885, -0.2082]]),
    x: Batch(
           c: tensor([[-1.8615,  1.6310,  0.0339,  0.1787],
                      [ 0.1815,  1.4565,  1.5177,  1.9505],
                      [ 1.0834,  2.7347, -0.6714, -0.6708]]),
       ),
)

In [13]:
%timeit b.a = new_value

543 ns ± 7.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

883 ns ± 0.33 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 28.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

148 µs ± 304 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

150 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f89938714c0>
├── 'a' --> tensor([[[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]],
│           
│                   [[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]],
│           
│                   [[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]],
│           
│                   [[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]],
│           
│                   [[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]],
│           
│                   [[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]],
│           
│                   [[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]],
│           
│                   [[-0.5418,  1.7838, -0.7986],
│                    [-1.1647,  0.0565,  0.3745]]])
└── 'x' --> <FastTreeValue 0x7f89938719d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.5 µs ± 52.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f89938717f0>
├── 'a' --> tensor([[-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745],
│                   [-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745],
│                   [-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745],
│                   [-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745],
│                   [-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745],
│                   [-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745],
│                   [-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745],
│                   [-0.5418,  1.7838, -0.7986],
│                   [-1.1647,  0.0565,  0.3745]])
└── 'x' --> <FastTreeValue 0x7f8993871490>
    └── 'c' --> tensor([[-1.8615,  1.6310,  0.0339,  0.1787],
                        [ 0.1815,  1.4565,  1.5177,  1.9505],
                 

In [23]:
%timeit t_cat(trees)

31.3 µs ± 66.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.2 µs ± 61 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.8615,  1.6310,  0.0339,  0.1787],
                       [ 0.1815,  1.4565,  1.5177,  1.9505],
                       [ 1.0834,  2.7347, -0.6714, -0.6708]],
              
                      [[-1.8615,  1.6310,  0.0339,  0.1787],
                       [ 0.1815,  1.4565,  1.5177,  1.9505],
                       [ 1.0834,  2.7347, -0.6714, -0.6708]],
              
                      [[-1.8615,  1.6310,  0.0339,  0.1787],
                       [ 0.1815,  1.4565,  1.5177,  1.9505],
                       [ 1.0834,  2.7347, -0.6714, -0.6708]],
              
                      [[-1.8615,  1.6310,  0.0339,  0.1787],
                       [ 0.1815,  1.4565,  1.5177,  1.9505],
                       [ 1.0834,  2.7347, -0.6714, -0.6708]],
              
                      [[-1.8615,  1.6310,  0.0339,  0.1787],
                       [ 0.1815,  1.4565,  1.5177,  1.9505],
                       [ 1.0834,  2.7347, -0.6714, -0.6708]],

In [26]:
%timeit Batch.stack(batches)

84.3 µs ± 152 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.8615,  1.6310,  0.0339,  0.1787],
                      [ 0.1815,  1.4565,  1.5177,  1.9505],
                      [ 1.0834,  2.7347, -0.6714, -0.6708],
                      [-1.8615,  1.6310,  0.0339,  0.1787],
                      [ 0.1815,  1.4565,  1.5177,  1.9505],
                      [ 1.0834,  2.7347, -0.6714, -0.6708],
                      [-1.8615,  1.6310,  0.0339,  0.1787],
                      [ 0.1815,  1.4565,  1.5177,  1.9505],
                      [ 1.0834,  2.7347, -0.6714, -0.6708],
                      [-1.8615,  1.6310,  0.0339,  0.1787],
                      [ 0.1815,  1.4565,  1.5177,  1.9505],
                      [ 1.0834,  2.7347, -0.6714, -0.6708],
                      [-1.8615,  1.6310,  0.0339,  0.1787],
                      [ 0.1815,  1.4565,  1.5177,  1.9505],
                      [ 1.0834,  2.7347, -0.6714, -0.6708],
                      [-1.8615,  1.6310,  0.0339,  0.1787],
                   

In [28]:
%timeit Batch.cat(batches)

152 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

374 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
